What this notebook does is to obtain 2021 papers' doi from crossref based on title query. Then I checked the titles and dois in the query results. For those with incorrect results, I obtained their DOI from IEEE Xplore. 

In [1]:
from habanero import Crossref
import pandas as pd
pd.options.display.max_colwidth = 200
import csv
import urllib.parse
from scholarly import scholarly
import requests
import re

In [2]:
cr = Crossref()

In [3]:
## importing the title file
df = pd.read_csv("../../data/processed/titles_2021.csv")
titles = df.title.tolist()
len(titles)

170

In [4]:
## Check for duplicates; none duplicates here
len(list(set(titles)))

170

In [5]:
title_doi_dict_list = []

In [6]:
titles[1]

'IRVINE: A Design Study on Analyzing Correlation Patterns of Electrical Engines'

In [7]:
x = cr.works(query = titles[1])

In [8]:
for t in titles:
    try: 
        x = cr.works(query=t)
        first_item = x['message']['items'][0]
        cr_title = first_item['title'][0]
        cr_doi = first_item['DOI']
        cr_year = first_item['indexed']['date-parts'][0][0]
        title_doi_dict = {
            'title': t,
            'crossref title': cr_title,
            'crossref doi': cr_doi,
            'crossref year': cr_year,
        }
        title_doi_dict_list.append(title_doi_dict)
        print(f'{titles.index(t)} is done')
    except:
        print(f'{titles.index(t)} has something wrong')

0 is done
1 is done
2 is done
3 is done
4 is done
5 is done
6 is done
7 is done
8 is done
9 is done
10 is done
11 is done
12 is done
13 is done
14 is done
15 is done
16 is done
17 is done
18 is done
19 is done
20 is done
21 is done
22 is done
23 is done
24 is done
25 is done
26 is done
27 is done
28 is done
29 is done
30 is done
31 is done
32 is done
33 is done
34 is done
35 is done
36 is done
37 is done
38 is done
39 is done
40 is done
41 is done
42 is done
43 is done
44 is done
45 is done
46 is done
47 is done
48 is done
49 is done
50 is done
51 is done
52 is done
53 is done
54 is done
55 is done
56 is done
57 is done
58 is done
59 is done
60 is done
61 is done
62 is done
63 is done
64 is done
65 is done
66 is done
67 is done
68 is done
69 is done
70 is done
71 is done
72 is done
73 is done
74 is done
75 is done
76 is done
77 is done
78 is done
79 is done
80 is done
81 is done
82 is done
83 is done
84 is done
85 is done
86 is done
87 is done
88 is done
89 is done
90 is done
91 is don

In [9]:
title_doi_df = pd.DataFrame(title_doi_dict_list)

In [12]:
## find out papers with different titles AND the doi contains 10.1109
## I will examine papers whose DOI does not contain 10.1109 in the following
mismatch_df = title_doi_df[(title_doi_df.title.str.lower() != title_doi_df['crossref title'].str.lower()) & (
    title_doi_df['crossref doi'].str.contains('10.1109')
)]
mismatch_df
# shape: (27,4)

,title,crossref title,crossref doi,crossref year
4,AffectiveTDA: Using Topological Data Analysis for Improved Explainability in Affective Computing,AffectiveTDA: Using Topological Data Analysis to Improve Analysis and Explainability in Affective Computing,10.1109/tvcg.2021.3114784,2022
7,M^2Lens: Visualizing and Explaining Multimodal Models for Sentiment Analysis,M2Lens: Visualizing and Explaining Multimodal Models for Sentiment Analysis,10.1109/tvcg.2021.3114794,2022
29,VitaLITy: Promoting Serendipitous Discovery of Academic Literature with Transformers & Visual Analytics,VITALITY: Promoting Serendipitous Discovery of Academic Literature with Transformers &amp; Visual Analytics,10.1109/tvcg.2021.3114820,2022
37,From Jam Session to Concert Hall: Synchronous Communication and Collaboration Around Data in Organizations,From Jam Session to Recital: Synchronous Communication and Collaboration Around Data in Organizations,10.1109/tvcg.2021.3114760,2022
42,matExplorer: Visual Exploration on Predicting of Ionic Conductivity for Solid-State Electrolytes,matExplorer: Visual Exploration on Predicting Ionic Conductivity for Solid-state Electrolytes,10.1109/tvcg.2021.3114812,2022
47,Data Badges: Making an Academic Profile through a DIY wearable physicalisation,Data Badges: Making an Academic Profile Through a DIY Wearable Physicalization,10.1109/mcg.2020.3025504,2022
48,Move&Find: The value of kinesthetic experience in a casual data representation,Move&amp;Find: The Value of Kinaesthetic Experience in a Casual Data Representation,10.1109/mcg.2020.3025385,2022
49,Slave Voyages:reflections on data sculptures,Slave Voyages: Reflections on Data Sculptures,10.1109/mcg.2020.3025183,2022
50,Narrative Physicalisation: Supporting Interactive Engagement with Personal Data,Narrative Physicalization: Supporting Interactive Engagement With Personal Data,10.1109/mcg.2020.3025078,2022
56,KG4Vis: A Knowledge Graph-Based Approach for Visualization Recommendation,<i>KG4Vis:</i> A Knowledge Graph-Based Approach for Visualization Recommendation,10.1109/tvcg.2021.3114863,2022


In [13]:
mismatch_df.shape

(27, 4)

I found some papers with absolutely different titles:
<!-- 
```
- IRVINE: Using Interactive Clustering and Labeling to Analyze Correlation Patterns: A Design Study from the Manufacturing of Electrical Engines

- “Why did my AI agent lose?”: Visual Analytics for Scaling Up AAR/AI
``` -->

- Understanding Data Visualization Design Practice

<!-- - Interactive Graph Construction for Graph-Based Semi-Supervised Learning -->

- Visual Evaluation for Autonomous Driving

- Visual Cascade Analytics of Large-scale Spatiotemporal Data

- Understanding Missing Links in Bipartite Networks with MissBiN

(NOTE ON 2022-02-27: I'VE CORRECTED THESE TWO PAPERS' TITLES IN THE SCRIPT OF `GET_TITLES_2021.PY`. SO I PROBABLY WON'T SEE THESE TWO PAPERS LATER IN THIS ANLAYSIS)

(NOTE ON 2022-04-04: IF YOU SEE THE SOURCE CODE OF THIS CHUNK, YOU'LL SEE THE HISTORY. “Why did my AI agent lose?”: Visual Analytics for Scaling Up AAR/AI", this is a short paper, so I did not include it and that's why you are not able to see it here. It is simply no longer part of `processed/titles_2021.csv`. Also, in `get_titles_2021.py`, I've already corrected the title for `IRVIN:...`, so you are not able to see it here. )

<!-- 

```
The first two papers, in fact, are correct. They changed their titles after presenting at VIS 2021. For example, 'IRVINE: Using Interactive Clustering and Labeling to Analyze Correlation Patterns: A Design Study from the Manufacturing of Electrical Engines' is the title for presentation. Then, for publication, it changed its name to 'IRVINE: A Design Study on Analyzing Correlation Patterns of Electrical Engines'. Note that it is not that these are two different papers: The old title is obsolete. The title on CrossRef is updated. For this paper, the crossref doi matches with that on ieee: https://ieeexplore.ieee.org/abstract/document/9552903

Another paper that changes its name is "“Why did my AI agent lose?”: Visual Analytics for Scaling Up AAR/AI". The title on CrossRef is the updated one. CrossRef doi also matches with the one on IEEE: https://ieeexplore.ieee.org/abstract/document/9623268

``` -->

In [14]:
# If their DOI does not contain IEEE DOI Prefox, 10.1109, then it's highly possible that 
#.   they are not the correct results. 
cr_failed_titles_df = title_doi_df[~title_doi_df['crossref doi'].str.contains('10.1109')]
cr_failed_titles_df
## 22 papers' doi don't contain '10.1109'
# among these 22 papers, 6 are indeed vis papers, just that theier doi is osf

,title,crossref title,crossref doi,crossref year
15,Interactive Dimensionality Reduction for Comparative Analysis,Dimensionality Reduction for Interactive Visual Clustering,10.4018/978-1-60960-102-7.ch004,2022
16,Real-Time Denoising of Volumetric Path Tracing for Direct Volume Rendering,Cinematic Rendering in UE4 with Real-Time Ray Tracing and Denoising,10.1007/978-1-4842-4427-2_19,2022
20,3D Virtual Pancreatography,Virtual Reality and Java 3D,10.4018/978-1-59904-789-8.ch001,2022
24,Rethinking the Ranks of Visual Channels,Rethinking the ranks of visual channels,10.31219/osf.io/n7kxu,2022
26,Modeling Just Noticeable Differences in Charts,Matching just-noticeable differences,10.1093/acprof:oso/9780198523420.003.0008,2022
46,What we talk about when we talk about data physicality,WHAT WE TALK ABOUT WHEN WE TALK ABOUT CITIES (AND LOVE),10.2307/j.ctv8bt3h9.2,2022
90,Natural Language to Visualization by Neural Machine Translation,Analysis and Visualization,10.1017/9781108608480.018,2022
101,Gosling: A Grammar-based Toolkit for Scalable and Interactive Genomics Data Visualization,Gosling: A Grammar-based Toolkit for Scalable and Interactive Genomics Data Visualization,10.31219/osf.io/6evmb,2022
104,Interactive Graph Construction for Graph-Based Semi-Supervised Learning,Graph-Based Semi-Supervised Learning,10.1007/978-3-031-01548-9_5,2022
112,Differentiable Direct Volume Rendering,Occlusion-resolving direct volume rendering,10.14711/thesis-b1054470,2022


In [15]:
cr_failed_titles = cr_failed_titles_df.title.tolist()
len(cr_failed_titles)

23

In [16]:
cr_failed_titles

['Interactive Dimensionality Reduction for Comparative Analysis',
 'Real-Time Denoising of Volumetric Path Tracing for Direct Volume Rendering',
 '3D Virtual Pancreatography',
 'Rethinking the Ranks of Visual Channels',
 'Modeling Just Noticeable Differences in Charts',
 'What we talk about when we talk about data physicality',
 'Natural Language to Visualization by Neural Machine Translation',
 'Gosling: A Grammar-based Toolkit for Scalable and Interactive Genomics Data Visualization',
 'Interactive Graph Construction for Graph-Based Semi-Supervised Learning',
 'Differentiable Direct Volume Rendering',
 'Interactive Visual Exploration of Longitudinal Historical Career Mobility Data',
 'A State-of-the-Art Survey of Tasks for Tree Design and Evaluation with a Curated Task Dataset',
 'STRATISFIMAL LAYOUT: A Modular Optimization Model for Laying Out Layered Node-link Network Visualizations',
 'A Design Space for Applying the Freytag’s Pyramid Structure to Data Stories',
 'Interactive Data

In [18]:
# For 23 titles in cr_failed_titles, I manually obtained their DOI from IEEE Xplore
manual_title_doi_dict = {
    'Interactive Dimensionality Reduction for Comparative Analysis': '10.1109/TVCG.2021.3114807',
    'Real-Time Denoising of Volumetric Path Tracing for Direct Volume Rendering': '10.1109/TVCG.2020.3037680',
    '3D Virtual Pancreatography': '10.1109/TVCG.2020.3020958',
    'Rethinking the Ranks of Visual Channels': '10.1109/TVCG.2021.3114684',
    'Modeling Just Noticeable Differences in Charts': '10.1109/TVCG.2021.3114874',
    'What we talk about when we talk about data physicality': '10.1109/MCG.2020.3024146',
    'Natural Language to Visualization by Neural Machine Translation': '10.1109/TVCG.2021.3114848',
    'Gosling: A Grammar-based Toolkit for Scalable and Interactive Genomics Data Visualization': '10.1109/TVCG.2021.3114876',
    'Interactive Graph Construction for Graph-Based Semi-Supervised Learning': '10.1109/TVCG.2021.3084694',
    'Differentiable Direct Volume Rendering': '10.1109/TVCG.2021.3114769',
    'Interactive Visual Exploration of Longitudinal Historical Career Mobility Data': '10.1109/TVCG.2021.3067200',
    'A State-of-the-Art Survey of Tasks for Tree Design and Evaluation with a Curated Task Dataset': '10.1109/TVCG.2021.3064037',
    'STRATISFIMAL LAYOUT: A Modular Optimization Model for Laying Out Layered Node-link Network Visualizations': '10.1109/TVCG.2021.3114756',
    'A Design Space for Applying the Freytag’s Pyramid Structure to Data Stories': '10.1109/TVCG.2021.3114774',
    'Interactive Data Comics': '10.1109/TVCG.2021.3114849',
    'Examining Effort in 1D Uncertainty Communication Using Individual Differences in Working Memory and NASA-TLX': '10.1109/TVCG.2021.3114803',
    'Visualization Equilibrium': '10.1109/TVCG.2021.3114842',
    'Visual Analysis of Hyperproperties for Understanding Model Checking Results': '10.1109/TVCG.2021.3114866',
    'COVID-view: Diagnosis of COVID-19 using Chest CT': '10.1109/TVCG.2021.3114851',
    'ThreadStates: State-based Visual Analysis of Disease Progression': '10.1109/TVCG.2021.3114840',
    'Loon: Using Exemplars to Visualize Large Scale Microscopy Data': '10.1109/TVCG.2021.3114766',
    'Scalable Scalable Vector Graphics: Automatic Translation of Interactive SVGs to a Multithread VDOM for Fast Rendering': '10.1109/TVCG.2021.3059294',
    'Probabilistic Data-Driven Sampling via Multi-Criteria Importance Analysis': '10.1109/TVCG.2020.3006426',
}

In [19]:
list(manual_title_doi_dict.keys()) == cr_failed_titles

True

what the above chunk checked was whether I've included all titles in `cr_failed_titles`. It is false here because Crossref is constantly changing. In `02-check-title-completedness`, I've made sure that every title in in `data/processed/titles_2021.csv` appeared in `data/processed/vispubdata_plus.csv`. Therefore, when we processed data, we had not problem finding doi for all 170 VIS 2021 papers. 